In [2]:
import argparse
import math
from mpi4py import MPI

def cos_function(x):
    """Returns the cosine of x."""
    return math.cos(x)

def trapezoid_rule(func, lower_bound, upper_bound, *, num_subdivisions=256):
    """Calculate the integral using the trapezoidal rule.
    
    Args:
        func: The function to integrate.
        lower_bound: The lower bound of integration.
        upper_bound: The upper bound of integration.
        num_subdivisions: The number of subdivisions (default 256).
    
    Returns:
        The approximate integral.
    """
    step_size = (upper_bound - lower_bound) / num_subdivisions
    integral = (func(lower_bound) + func(upper_bound)) * step_size / 2
    for i in range(1, num_subdivisions):
        x_i = lower_bound + step_size * i
        integral += func(x_i)
    return integral

def distribute_bounds(lower_bound, upper_bound, num_processes):
    """Distribute the integration bounds across processes."""
    chunk_size = (upper_bound - lower_bound) / num_processes
    bounds = [(lower_bound + i * chunk_size, lower_bound + (i + 1) * chunk_size) for i in range(num_processes)]
    return bounds

def main():
    """Main function to handle the distributed numerical integration."""
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    
    # Define integration parameters in rank 0
    if rank == 0:
        parser = argparse.ArgumentParser(description='Numerical integral using the trapezoid rule')
        parser.add_argument("-a", help="The lower bound of the definite integral", type=float, required=True)
        parser.add_argument("-b", help="The upper bound of the definite integral", type=float, required=True)
        parser.add_argument("-n", help="The number of steps for the numerical approximation", type=int, default=256)
        args = parser.parse_args()
        
        lower_bound = args.a
        upper_bound = args.b
        num_steps = args.n
        
        # Divide the bounds across processes
        bounds = distribute_bounds(lower_bound, upper_bound, size)
        local_num_steps = num_steps // size  # Approximate step count per process
    else:
        bounds = None
        local_num_steps = None

    # Scatter bounds and local step count to each process
    local_bound = comm.scatter(bounds, root=0)
    local_num_steps = comm.bcast(local_num_steps, root=0)
    
    # Calculate the local integral for the assigned bounds
    local_lower_bound, local_upper_bound = local_bound
    local_integral = trapezoid_rule(cos_function, local_lower_bound, local_upper_bound, num_subdivisions=local_num_steps)

    # Gather all local integrals to rank 0
    all_integrals = comm.gather(local_integral, root=0)
    
    if rank == 0:
        # Sum the local integrals to get the total integral
        total_integral = sum(all_integrals) * ((upper_bound - lower_bound) / num_steps)
        print(f"Approximate integral of cos(x) from {lower_bound} to {upper_bound} with {num_steps} steps: {total_integral}")

if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] -a A -b B [-n N]
ipykernel_launcher.py: error: the following arguments are required: -a, -b


SystemExit: 2

/homes/zhe/Desktop/programming/p5/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
